<a href="https://colab.research.google.com/github/ganeshnikhil/J.A.R.V.I.S.2.0/blob/main/Train/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth[torch] transformers accelerate vllm


In [ ]:
import unsloth
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False
)


==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import Dataset

# Example prompt-response pair
samples = [
    {
        "input": "User Query: What's the weather in London?",
        "output": """[
            {
                "name": "weather_report",
                "parameters": {"location": "London"}
            }
        ]"""
    },
    {
        "input": "User Query: Open Chrome for me",
        "output": """[
            {
                "name": "app_runner",
                "parameters": {"app_name": "chrome"}
            }
        ]"""
    },
    {
        "input": "User Query: Open YouTube.",
        "output": """[
            {
                "name": "app_runner",
                "parameters": {"app_name": "youtube"}
            }
        ]"""
    },
    {
        "input": "User Query: Get news headlines.",
        "output": """[
            {
                "name": "news_headlines",
                "parameters": {}
            }
        ]"""
    },
    {
        "input": "User Query: Get stock data for NASDAQ.",
        "output": """[
            {
                "name": "get_stock_data",
                "parameters": {"exchange": "NASDAQ"}
            }
        ]"""
    },
    {
        "input": "User Query: Search YouTube for NASDAQ analysis.",
        "output": """[
            {
                "name": "search_youtube",
                "parameters": {"topic": "NASDAQ analysis"}
            }
        ]"""
    },
    {
        "input": "User Query: Who will win the FIFA World Cup in 2030?",
        "output": """[
            {
                "name": "duckgo_search",
                "parameters": {"query": "FIFA World Cup winners in 2030?"}
            }
        ]"""
    },
    {
        "input": "User Query: Open Instagram web.",
        "output": """[
            {
                "name": "app_runner",
                "parameters": {"app_name": "instagram"}
            }
        ]"""
    },
    {
        "input": "User Query: Who is the current president of Brazil?",
        "output": """[
            {
                "name": "duckgo_search",
                "parameters": {"query": "current president of Brazil?"}
            }
        ]"""
    },
    {
        "input": "User Query: Get stock data for NASDAQ and then Open Instagram web.",
        "output": """[
            {
                "name": "get_stock_data",
                "parameters": {"exchange": "NASDAQ"}
            },
            {
                "name": "app_runner",
                "parameters": {"app_name": "instagram"}
            }
        ]"""
    },
    {
        "input": "User Query: Get news headlines and then Open YouTube.",
        "output": """[
            {
                "name": "news_headlines",
                "parameters": {}
            },
            {
                "name": "app_runner",
                "parameters": {"app_name": "youtube"}
            }
        ]"""
    },
    {
        "input": "User Query: Tell me a joke.",
        "output": """[
            {
                "name": "send_to_ai",
                "parameters": {"prompt": "tell a joke."}
            }
        ]"""
    },
    {
        "input": "User Query: book a ticket flight",
        "output": """[]"""
    },
    {
        "input": "User Query: What's the weather like in Tokyo?",
        "output": """[
            {
                "name": "weather_report",
                "parameters": {"location": "Tokyo"}
            }
        ]"""
    },
    {
        "input": "User Query: Give me the stock data for NASDAQ.",
        "output": """[
            {
                "name": "get_stock_data",
                "parameters": {"exchange": "NASDAQ"}
            }
        ]"""
    },
    {
        "input": "User Query: Search YouTube for machine learning tutorials.",
        "output": """[
            {
                "name": "search_youtube",
                "parameters": {"topic": "machine learning tutorials"}
            }
        ]"""
    },
    {
        "input": "User Query: Download this YouTube video: https://youtu.be/xyz123",
        "output": """[
            {
                "name": "yt_download",
                "parameters": {"url": "https://youtu.be/xyz123"}
            }
        ]"""
    },
    {
        "input": "User Query: Tell me a short story.",
        "output": """[
            {
                "name": "send_to_ai",
                "parameters": {"prompt": "Tell me a short story."}
            }
        ]"""
    },
    {
        "input": "User Query: Launch Spotify.",
        "output": """[
            {
                "name": "app_runner",
                "parameters": {"app_name": "Spotify"}
            }
        ]"""
    },
    {
        "input": "User Query: Search privately for best travel destinations.",
        "output": """[
            {
                "name": "private_mode",
                "parameters": {"query": "best travel destinations"}
            }
        ]"""
    },
    {
        "input": "User Query: Call Mom.",
        "output": """[
            {
                "name": "make_a_call",
                "parameters": {"contact_name": "Mom"}
            }
        ]"""
    },
    {
        "input": "User Query: Send an email to alex@example.com saying Hello with subject Greetings.",
        "output": """[
            {
                "name": "send_email",
                "parameters": {
                    "recipient": "alex@example.com",
                    "subject": "Greetings",
                    "body": "Hello"
                }
            }
        ]"""
    },
    {
        "input": "User Query: Search for AI in healthcare on DuckDuckGo.",
        "output": """[
            {
                "name": "duckgo_search",
                "parameters": {"query": "AI in healthcare"}
            }
        ]"""
    },
    {
        "input": "User Query: I want to know the weather in Mumbai and also get the latest news.",
        "output": """[
            {
                "name": "weather_report",
                "parameters": {"location": "Mumbai"}
            },
            {
                "name": "news_headlines",
                "parameters": {}
            }
        ]"""
    },
    {
        "input": """User Query: Play some music.""",
        "output": """[
            {"name": "app_runner",
            "parameters": {"app_name": "Spotify"}}]"""
    },
    {
        "input": """User Query: Call my friend rohan.""",
        "output": """[
            {"name": "make_a_call",
            "parameters": {"contact_name": "rohan"}}]"""
    },
    {
        "input": """User Query: what is addition of 2+2.""",
        "output": """[
            {"name": "send_to_ai",
            "parameters": {"prompt": "addition of 2+2 is"}}]"""
    },
    {
        "input": "User Query: Who will win the FIFA World Cup in 2030?",
        "output": """[
            {
                "name": "duckgo_search",
                "parameters": {"query": "FIFA World Cup winners in 2030?"}
            }
        ]"""
    },
    {
      "input": "User Query: current stock price of Tesla",
      "output": """[
        {
          "name": "duckgo_search",
          "parameters": {
            "query": "current stock price of Tesla"
          }
        }
      ]"""
    },
    {
      "input": "User Query: how to stay motivated?",
      "output": """[
        {
          "name": "send_to_ai",
          "parameters": {
            "prompt": "how to stay motivated?"
          }
        }
      ]"""
    },
    {
      "input": "User Query: Get information about the 2025 Indian budget.",
      "output": """[
        {
          "name": "duckgo_search",
          "parameters": {
            "query": "2025 Indian budget"
          }
        }
      ]"""
    },
    {
      "input": "User Query: Show me current COVID-19 cases in India.",
      "output": """[
        {
          "name": "duckgo_search",
          "parameters": {
            "query": "current COVID-19 cases in India"
          }
        }
      ]"""
    },
    {
      "input": "User Query: I want to know the weather in Mumbai and also get the latest news.",
      "output": """[
        {
          "name": "weather_report",
          "parameters": {
            "location": "Mumbai"
          }
        },
        {
          "name": "news_headlines",
          "parameters": {}
        }
      ]"""
    },
    {
      "input": "User Query: What’s the weather in Delhi and who won the last IPL match?",
      "output": """[
        {
          "name": "weather_report",
          "parameters": {
            "location": "Delhi"
          }
        },
        {
          "name": "duckgo_search",
          "parameters": {
            "query": "last IPL match winner"
          }
        }
      ]"""
    },
    {
      "input": "User Query: Send an email to my friend and also check today’s weather in Dehradun.",
      "output": """[
        {
          "name": "email_sender",
          "parameters": {
            "recipient": "my friend",
            "content": ""
          }
        },
        {
          "name": "weather_report",
          "parameters": {
            "location": "Dehradun"
          }
        }
      ]"""
    },
  {
    "input": "User Query: Tell me a joke and find current stock price of Apple.",
    "output": """[
      {
        "name": "send_to_ai",
        "parameters": {
          "prompt": "tell me a joke"
        }
      },
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "current stock price of Apple"
        }
      }
    ]"""
  },
  {
    "input": "User Query: Explain blockchain and show news about cryptocurrency.",
    "output": """[
      {
        "name": "send_to_ai",
        "parameters": {
          "prompt": "Explain blockchain"
        }
      },
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "cryptocurrency news"
        }
      }
    ]"""
  },
  {
  "input": "User Query: I want to know the weather in Mumbai and also get the latest news.",
  "output": """[
    {
      "name": "weather_report",
      "parameters": {
        "location": "Mumbai"
      }
    },
    {
      "name": "news_headlines",
      "parameters": {}
    }
  ]"""
  },
  {
    "input": "User Query: What’s the weather in Delhi and who won the last IPL match?",
    "output": """[
      {
        "name": "weather_report",
        "parameters": {
          "location": "Delhi"
        }
      },
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "last IPL match winner"
        }
      }
    ]"""
  },
  {
    "input": "User Query: Send an email to my friend and also check today’s weather in Dehradun.",
    "output": """[
      {
        "name": "email_sender",
        "parameters": {
          "recipient": "my friend",
          "content": ""
        }
      },
      {
        "name": "weather_report",
        "parameters": {
          "location": "Dehradun"
        }
      }
    ]"""
  },
  {
    "input": "User Query: Tell me a joke and find current stock price of Apple.",
    "output": """[
      {
        "name": "send_to_ai",
        "parameters": {
          "prompt": "tell me a joke"
        }
      },
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "current stock price of Apple"
        }
      }
    ]"""
  },
  {
  "input": "User Query: Give me the weather in Chennai, latest news, and explain quantum computing.",
  "output": """[
    {
      "name": "weather_report",
      "parameters": {
        "location": "Chennai"
      }
    },
    {
      "name": "news_headlines",
      "parameters": {}
    },
    {
      "name": "send_to_ai",
      "parameters": {
        "prompt": "Explain quantum computing"
      }
    }
  ]"""
  },
  {
    "input": "User Query: What's the weather in Bangalore, and search DuckDuckGo for best programming languages in 2025.",
    "output": """[
      {
        "name": "weather_report",
        "parameters": {
          "location": "Bangalore"
        }
      },
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "best programming languages in 2025"
        }
      }
    ]"""
  },
  {
    "input": "User Query: Tell me a motivational quote and today's weather in Hyderabad.",
    "output": """[
      {
        "name": "send_to_ai",
        "parameters": {
          "prompt": "give me a motivational quote"
        }
      },
      {
        "name": "weather_report",
        "parameters": {
          "location": "Hyderabad"
        }
      }
    ]"""
  },
  {
    "input": "User Query: Give me headlines and explain the difference between AI and ML.",
    "output": """[
      {
        "name": "news_headlines",
        "parameters": {}
      },
      {
        "name": "send_to_ai",
        "parameters": {
          "prompt": "difference between AI and ML"
        }
      }
    ]"""
  },
  {
    "input": "User Query: What’s the current AQI in Kolkata and give a tip to reduce stress.",
    "output": """[
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "current AQI in Kolkata"
        }
      },
      {
        "name": "send_to_ai",
        "parameters": {
          "prompt": "give a tip to reduce stress"
        }
      }
    ]"""
  },
  {
    "input": "User Query: I want news, today's weather in Lucknow, and also search for best laptops under 60000.",
    "output": """[
      {
        "name": "news_headlines",
        "parameters": {}
      },
      {
        "name": "weather_report",
        "parameters": {
          "location": "Lucknow"
        }
      },
      {
        "name": "duckgo_search",
        "parameters": {
          "query": "best laptops under 60000"
        }
      }
    ]"""
  }
]

def formatting_prompts(example):
    return f"<|user|>\n{example['input']}\n<|assistant|>\n{example['output']}"

dataset = Dataset.from_list(samples)
dataset = dataset.map(lambda x: {"text": formatting_prompts(x)})


Map:   0%|          | 0/47 [00:00<?, ? examples/s]

In [ ]:
from unsloth import FastLanguageModel

model = FastLanguageModel.get_peft_model(
    model,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
from trl import SFTTrainer, SFTConfig
import os
# os.environ['TRITON_DISABLE_LINE_INFO'] = '1' # Optional, tested with and without
os.environ['TRITON_JIT_DISABLE_OPT'] = '1' # Likely the most critical change
os.environ["TRITON_DISABLE_LINE_INFO"] = '1'

# 🔧 Ensure use_cache is off when not using checkpointing
model.config.use_cache = False  # <-- This is the key line

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    dataset_text_field = "text",  # <- This goes here, not inside SFTConfig
    args = SFTConfig(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    num_train_epochs = 3,              # Instead of just steps
    learning_rate = 1e-4,              # Slightly lower for stability
    max_grad_norm = 1.0,
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",      # Can help smoothen learning
    seed = 3407,
    fp16 = True,                       # Enable if supported
    gradient_checkpointing = True,     # Helps with memory
  )
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/47 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47 | Num Epochs = 4 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,636,096/1,000,000,000 (0.56% trained)


Step,Training Loss
1,0.711300
2,0.736700
3,0.766500
4,0.734700
5,0.655100
6,0.681200
7,0.595200
8,0.564500
9,0.589300
10,0.529800


TrainOutput(global_step=20, training_loss=0.5171504944562912, metrics={'train_runtime': 31.9226, 'train_samples_per_second': 5.012, 'train_steps_per_second': 0.627, 'total_flos': 69109854142464.0, 'train_loss': 0.5171504944562912})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path = "/content/drive/MyDrive/llm_models/lora_model"

# Create directory if it doesn't exist
import os
os.makedirs(save_path, exist_ok=True)

# Save model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/llm_models/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/llm_models/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/llm_models/lora_model/chat_template.jinja',
 '/content/drive/MyDrive/llm_models/lora_model/tokenizer.json')

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_id = "unsloth/Llama-3.2-1B-Instruct"
peft_model_dir = "/content/drive/MyDrive/llm_models/lora_model" # Replace with your path

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, peft_model_dir)


In [ ]:
# Step 1: Save locally to /content (faster)
# Step 1: Merge LoRA weights into the base model
model = model.merge_and_unload()

# Step 2: Save the merged model locally (in full precision)
local_path = "/content/jarvis-3.2-1B"
model.save_pretrained(local_path, safe_serialization=True)  # Use safetensors
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained(local_path)

# Step 3: (Optional) Copy to Google Drive if needed
drive_path = "/content/drive/MyDrive/jarvis-3.2-1B"
!cp -r {local_path} {drive_path}
#/content/jarvis-3.2-1B/model.safetensors

In [ ]:
# %%
import shutil
import os

source_dir = "/content/drive/MyDrive/jarvis-3.2-1B"
destination_dir = "/content/drive/MyDrive/jarvis-3.2-1B" # Choose a path in your Drive

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy the entire directory from Colab to Google Drive
shutil.copytree(source_dir, destination_dir)

print(f"Model saved to Google Drive at: {destination_dir}")